# Loading the libraries

In [1]:
import numpy as np
import openseespy.opensees as ops
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import bisect
import math
import os
from openpyxl import load_workbook
import random

# Functions

## Function to Initialize the Model

In [2]:
# 1. Initialize the model
def initialize_model():
    ''' Initialize the model and wipe the previous session. '''
    ops.wipe()  # Clear any previous model
    ops.model('basic', '-ndm', 3, '-ndf', 6)  # 3D model with 6 DOFs per node


In [3]:
initialize_model()

 ## Function to Define Pipe Material and create Nodes

In [4]:
# 2. Define Pipe Material (Elastic Isotropic for 3D Pipe)
def define_pipe_material(E_pipe, nu_pipe, rho_pipe):
    ''' 
    Define pipe material properties: Young's modulus, Poisson's ratio, and density.
    '''
    ops.nDMaterial('ElasticIsotropic', 1, E_pipe, nu_pipe, rho_pipe)

In [5]:
# # 3. Creating the pipe nodes
# def create_pipe_nodes(pipe_radius, pipe_length):
#     global pipe_nodes
#     ''' Create pipe nodes in a 3D space. '''
#     half_length = pipe_length / 2
#     pipe_nodes = [
#         [-half_length, -pipe_radius, -pipe_radius],
#         [half_length, -pipe_radius, -pipe_radius],
#         [half_length, pipe_radius, -pipe_radius],
#         [-half_length, pipe_radius, -pipe_radius],
#         [-half_length, -pipe_radius, pipe_radius],
#         [half_length, -pipe_radius, pipe_radius],
#         [half_length, pipe_radius, pipe_radius],
#         [-half_length, pipe_radius, pipe_radius]
#     ]
#     return pipe_nodes

In [6]:
# 3. Creating the pipe nodes
def create_pipe_nodes(pipe_radius, pipe_length):
    global pipe_nodes
    ''' Create pipe nodes in a 3D space and define them in OpenSees. '''
    half_length = pipe_length / 2
    pipe_nodes = [
        [-half_length, -pipe_radius, -pipe_radius],
        [half_length, -pipe_radius, -pipe_radius],
        [half_length, pipe_radius, -pipe_radius],
        [-half_length, pipe_radius, -pipe_radius],
        [-half_length, -pipe_radius, pipe_radius],
        [half_length, -pipe_radius, pipe_radius],
        [half_length, pipe_radius, pipe_radius],
        [-half_length, pipe_radius, pipe_radius]
    ]
    
    # Define nodes in OpenSees with unique identifiers
    for i in range(len(pipe_nodes)):
        node_id = i + 1  # Node IDs start from 1
        coords = pipe_nodes[i]
        ops.node(node_id, coords[0], coords[1], coords[2])
        print(f"Node {node_id} created at {coords}")

    return pipe_nodes

In [7]:
# def create_pipe_nodes(pipe_radius, pipe_length):
#     global pipe_nodes
#     ''' Create pipe nodes in a 3D space and define them in OpenSees. '''
#     half_length = pipe_length / 2
#     pipe_nodes = [
#         [-half_length, -pipe_radius, -pipe_radius],
#     ]
    
#     # Define nodes in OpenSees with unique identifiers
#     for i in range(len(pipe_nodes)):
#         node_id = i + 1  # Node IDs start from 1
#         coords = pipe_nodes[i]
#         ops.node(node_id, coords[0], coords[1], coords[2])
#         print(f"Node {node_id} created at {coords}")

#     return pipe_nodes

In [8]:
pipe_radius = 1.27   
pipe_length = 100
pipe_nodes = create_pipe_nodes(pipe_radius, pipe_length)

Node 1 created at [-50.0, -1.27, -1.27]
Node 2 created at [50.0, -1.27, -1.27]
Node 3 created at [50.0, 1.27, -1.27]
Node 4 created at [-50.0, 1.27, -1.27]
Node 5 created at [-50.0, -1.27, 1.27]
Node 6 created at [50.0, -1.27, 1.27]
Node 7 created at [50.0, 1.27, 1.27]
Node 8 created at [-50.0, 1.27, 1.27]


# Functions to Create and Update Soil Materials

In [9]:
#4. Function to update soil material properties (E_soil) based on temperature
def update_soil_material(prev_temp, temp_E_pairs, nu_soil):
    ''' Update Young's modulus of soil based on temperature. '''
    temperatures, E_values = zip(*temp_E_pairs)
    
    if prev_temp <= temperatures[0]:
        E_soil = E_values[0]
    elif prev_temp >= temperatures[-1]:
        E_soil = E_values[-1]
    else:
        idx = bisect.bisect_right(temperatures, prev_temp)
        t0, t1 = temperatures[idx-1], temperatures[idx]
        E0, E1 = E_values[idx-1], E_values[idx]
        E_soil = E0 + (E1 - E0) * (prev_temp - t0) / (t1 - t0)
    
    return E_soil  # Return the calculated Young's modulus for the given temperature

In [10]:
# 4. Define Soil Material (Elastic with Freeze-Thaw)
def define_soil_material(E_soil_frozen, E_soil_thawed, nu_soil):
    ''' Define frozen and thawed soil materials using Elastic material model. '''
    ops.uniaxialMaterial('Elastic', 9, E_soil_frozen)  # Frozen soil material
    ops.uniaxialMaterial('Elastic', 10, E_soil_thawed)  # Thawed soil material

In [11]:
# 5. Creating the soil nodes
def create_soil_nodes(num_nodes_per_side, soil_size):
    ''' Create 3D grid of soil nodes. '''
    node_counter = 9
    soil_nodes = {}
    spacing = soil_size / (num_nodes_per_side - 1)
    
    for i in range(num_nodes_per_side):
        for j in range(num_nodes_per_side):
            for k in range(num_nodes_per_side):
                x = i * spacing - soil_size / 2
                y = j * spacing - soil_size / 2
                z = k * spacing - soil_size / 2
                ops.node(node_counter, x, y, z)
                soil_nodes[node_counter] = (x, y, z)
                node_counter += 1
    return soil_nodes

In [12]:
num_nodes_per_side = 8
soil_size = 1000

soil_nodes = create_soil_nodes(num_nodes_per_side, soil_size)

## Function to Create Pipe_Soil Interaction

In [13]:
# 6. Find the closest soil node to the pipe node
def find_closest_soil_node(pipe_node, soil_nodes):
    ''' Find the closest soil node to the given pipe node. '''
    closest_node = None
    min_distance = float('inf')
    for node_id, coords in soil_nodes.items():
        distance = sum((p - s)**2 for p, s in zip(pipe_node, coords))**0.5
        if distance < min_distance:
            min_distance = distance
            closest_node = node_id
    return closest_node


In [14]:
# 7. Pipe Soil Interaction
def create_pipe_soil_interaction_elements(pipe_nodes, soil_nodes, spring_stiffness, area=1.0):
    ''' Create pipe-soil interaction elements (springs between pipe and soil). '''
    element_counter = 1
    ops.uniaxialMaterial('Elastic', 1, spring_stiffness)  # Material with tag 1 and spring stiffness
    
    for i, pipe_node in enumerate(pipe_nodes):
        closest_soil_node = find_closest_soil_node(pipe_node, soil_nodes)
        
        pipe_node_id = i + 1
        soil_node_id = closest_soil_node
        
        try:
            ops.element('truss', element_counter, pipe_node_id, soil_node_id, area, 1)
        except Exception as e:
            print(f"Error creating truss element: {e}")
        
        element_counter += 1

## Funtion to Calculate the displacement of pipe

In [15]:
# 10. Calculating the displacment
def calculate_displacement(pressure, pipe_radius, E_pipe, pipe_thickness):
    # Hoop stress calculation
    hoop_stress = (pressure * pipe_radius) / pipe_thickness
    
    # Radial displacement calculation
    displacement = (pressure * pipe_radius**2) / (E_pipe * pipe_thickness)
    
    return displacement

## Function to Calculate Pressure at Pipe Node

In [41]:
# 8. Calculate Pressure at Pipe Node
def calculate_pressure_at_node(temp, prev_temp, radius, burst_pressure, K_water=2.2e9, alpha_pipe=0.000012):
    ''' Calculate pressure at pipe node based on temperature change. '''
    initial_volume = math.pi * radius ** 2
    new_radius = radius * (1 + alpha_pipe * (temp - prev_temp))  
    new_volume = math.pi * new_radius ** 2
    volume_change = new_volume - initial_volume

    total_pressure = burst_pressure  # Start with the burst pressure as a base

    # Temperature-induced pressure changes
    alpha_water = 0.00021
    temp_change = temp - prev_temp
    thermal_pressure = K_water * alpha_water * temp_change
    total_pressure += thermal_pressure

    # Freezing-related pressure changes
    if temp < 0:
        freezing_expansion_factor = 0.09
        freezing_volume_change = initial_volume * freezing_expansion_factor
        pressure_increase_freezing = K_water * (freezing_volume_change / initial_volume)
        total_pressure += pressure_increase_freezing

    # Thawing-related pressure changes
    if prev_temp < 0 and temp >= 0:
        thaw_pressure_factor = 1.5
        total_pressure *= thaw_pressure_factor

    return max(0, total_pressure)  # Ensure pressure is non-negative

In [42]:
def estimate_burst_location(pipe_nodes, burst_pressure, temp, prev_temp, pipe_radius, spring_stiffness, pipe_length, nu_soil, E_pipe, pipe_thickness):
    burst_results = []
    
    for i, pipe_node in enumerate(pipe_nodes):
        # Calculate the pressure at the current pipe node
        pressure_at_node = calculate_pressure_at_node(temp, prev_temp, pipe_radius, burst_pressure)
        
        # Calculate the new radius and new volume at this node
        initial_radius = pipe_radius  # Use the initial pipe radius
        alpha_pipe = 16.5e-6
        new_radius = initial_radius * (1 + alpha_pipe * (temp - prev_temp))  # Temperature-induced change in radius
        new_volume = math.pi * new_radius ** 2  # Calculate the new volume based on the new radius
        
        # Check if the pipe bursts at this node
        burst_pipe = pressure_at_node > burst_pressure
        
        # Update soil material properties based on temperature
        E_soil = update_soil_material(temp, temp_E_pairs, nu_soil)

        # Calculate displacement caused by pressure
        displacement = calculate_displacement(pressure_at_node, pipe_radius, E_pipe, pipe_thickness)
        
        # Append results for this node, including new_radius and new_volume
        burst_results.append({
            'pipe_node': pipe_node,
            'pressure_at_node': pressure_at_node,
            'burst_pressure': burst_pressure,
            'burst_occurred': burst_pipe,
            'E_soil': E_soil,
            'spring_stiffness': spring_stiffness,
            'displacement': displacement,
            'new_radius': new_radius,   # Store the new radius
            'new_volume': new_volume    # Store the new volume
        })
    
    return burst_results


## Function to Save the Result

In [43]:
def save_results_to_excel(all_results, temp_values, prev_temp_values, burst_pressure, output_path):
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        # Create an initial dummy sheet to avoid the "no visible sheet" error
        dummy_df = pd.DataFrame(columns=['Temp', 'Prev Temp', 'Dummy'])
        dummy_df.to_excel(writer, sheet_name="Dummy", index=False)

        # Loop over all the results for each analysis cycle
        for i, burst_results in enumerate(all_results):
            # Get the temp and prev_temp for the current analysis cycle
            temp = temp_values[i]
            prev_temp = prev_temp_values[i]
            
            # Prepare data for this analysis cycle
            data = []
            for result in burst_results:
                # Access the dictionary keys to get values
                pipe_node = result['pipe_node']
                E_soil = result['E_soil']
                pressure_at_node = result['pressure_at_node']
                spring_stiffness_at_node = result['spring_stiffness']
                pipe_burst_pressure = result['burst_pressure']
                burst_occurred = result['burst_occurred']
                displacement = result['displacement'] 
                new_radius = result['new_radius']  # Get new_radius
                new_volume = result['new_volume']  # Get new_volume
                
                # Add the temp, prev_temp, and constant burst pressure values for this cycle
                data.append([pipe_node, E_soil, pressure_at_node, spring_stiffness_at_node, 
                             pipe_burst_pressure, burst_occurred, temp, prev_temp, burst_pressure, displacement, new_radius, new_volume])
            
            # Convert the results into a DataFrame
            df = pd.DataFrame(data, columns=[ 
                'Pipe Node', 'Soil Modulus (E_soil)', 'Pressure at Node', 'Spring Stiffness',
                'Pipe Burst Pressure', 'Burst Occurred', 'Temp', 'Prev Temp', 'Constant Burst Pressure', "Displacement", "New Radius", "New Volume"
            ])
            
            # Save each analysis cycle to a separate sheet
            df.to_excel(writer, sheet_name=f"Analysis_{i + 1}", index=False)


## Function to Run the Analysis for various Temp and Prev_Temp

In [44]:
# Create a list of temperature-E pairs
temp_E_pairs = [
    (-20, 1000000000.0),  # 1 GPa (frozen soil)
    (-10, 800000000.0),   # 800 MPa
    (-5, 600000000.0),    # 600 MPa
    (0, 400000000.0),     # 400 MPa
    (5, 200000000.0),     # 200 MPa
    (10, 100000000.0),    # 100 MPa
    (20, 50000000.0)      # 50 MPa (thawed soil)
]

In [45]:
# 11. Function to run analysis
def run_analysis():
    # Example temperature pairs for analysis
    temp_values = [-7, -9, -5, 0, 5, 10, 15, 20, 25, 30, -10, -5, 0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 24, 22, -15, 27, 38, 23, 33, 5, 25, -7,
                  -15, 30, 10, -6, 0, -8, 0, -11, 42, -12, -13, 34, 38, -15, 24, 5, 33, 49, 18, -12, 9, 24, -8, 43, 37, 10, 26, 25, 16, -5, 49, -11, 
                   50, -2, 38, -5, 44, 12, 37, 0, 50, 4, 7, -7, 13, -13, 41, 10, 40, 42, 9, -7, 45, -12, 26, 24, -15, 36, 41, 25, 46, 5, 27, -9, 50,
                   6, 41, 41, 13, -10, 17, 12, 40, -2, -9, 9, 15, 17, -8, 0, 13, 9, 20, 7, 15, 3, 36, 18, 3, 1, 46, 0, 13, 20, -1, -4, 41, 9, 5, 29]
    
    prev_temp_values = [-11, -1, 0, 5, 10, 15, 17, 20, 25, -5, 0, 3, 5, 10, 15, 20, 25, 30, 35, 27, 31, 40, 5, 4, 3, 43, 23, 2, -6, -14, -11, 8,
                       8, -9, -12, -3, -6, 15, 0, 22, 25, 38, 33, 25, 28, -1, 41, 19, 29, 26, 10, 35, 45, 10, 0, 33, 8, 43, 5, -15, 
                        -15, -9, 30, 47, 24, 50, 23, 33, 23, -7, 7, 18, 13, -4, 35, 7, 50, -2, 10, 18, 19, -13, 13, 34, 23, -11, 49, 
                        36, -2, 33, 17, -9, 45, -5, 41, 15, 4, 34, 34, 38, 49, -11, 38, 23, 36, -6, 22, 18, -11, 41, 
                        30, 34, 17, -10, -11, 35, 33, 10, 23, 40, 45, 32, 8, 19, -10, 1, -8, 28, -1, 18, -2, 2]
    
    all_results = []  # List to store results for all analysis cycles

    for temp, prev_temp in zip(temp_values, prev_temp_values):
        # Step 1: Define the analysis parameters
        # pipe_nodes = [1, 2, 3, 4, 5, 6, 7, 8]  # Example pipe nodes
        # soil_nodes = [1, 2, 3, 4, 5, 6, 7, 8]  # Example soil nodes
        burst_pressure = 345000000  # Example burst pressure
        pipe_radius = 1.27  # Example pipe radius
        spring_stiffness = 1.0e7  # Example spring stiffness
        pipe_length = 100  # Example pipe length
        nu_soil = 0.3  # Example Poisson's ratio for soil
        E_pipe = 117000000000 # in Gpa
        pipe_thickness = 0.2 #cm
        nu_pipe = 0.33
        rho_pipe = 8960

        # Step 2: Run analysis for each temperature and previous temperature pair
        burst_results = estimate_burst_location(pipe_nodes, burst_pressure, temp, prev_temp, pipe_radius, 
                                                spring_stiffness, pipe_length, nu_soil, E_pipe, pipe_thickness)
        # Collect the results
        all_results.append(burst_results)

    # Step 3: Save results to Excel
    output_path = 'burst_results_analyses.xlsx'
    save_results_to_excel(all_results, temp_values, prev_temp_values, burst_pressure, output_path)
    print("Results saved to:", output_path)



In [46]:
random_temp_values = [random.randint(-15, 50) for _ in range(100)]
random_prev_temp_values = [random.randint(-15, 50) for _ in range(100)]

print(random_temp_values)
print(random_prev_temp_values)

[17, -2, 32, 48, 42, 13, 38, 43, 28, 32, 40, 37, -7, 45, -4, -3, 20, 25, 27, -5, 33, 14, 47, -4, 49, -7, -13, 45, 43, 42, 49, 6, 4, -4, 42, -5, 42, 0, -13, 29, 8, 47, 31, 22, 2, -11, 18, 36, 7, 30, -6, 47, 21, 31, 17, -8, 17, 36, 0, 10, 21, 4, 19, 49, -10, 37, 1, 22, -9, 27, 1, 31, 13, 10, 5, 47, 17, 37, 1, 30, -8, 33, 15, 5, 24, -7, 1, 27, 6, 15, -1, 43, -2, 2, -5, 33, 37, 33, 28, 3]
[-12, -9, -3, 9, 48, 15, 41, 18, -5, -5, -2, -14, 17, 6, 4, 7, 23, 0, 39, 2, 34, 15, 17, 43, 34, 45, 27, -5, 7, 39, 26, 4, 14, -9, 31, 8, 15, 29, -13, 45, 14, -8, 8, 13, 19, 37, -5, 39, -14, 41, 20, 36, 38, 37, -2, 50, -12, 46, 11, 8, -1, 18, -6, -7, -5, 36, -9, 36, 28, 13, 17, 38, 28, 10, 47, 39, 27, 7, 6, 17, 31, 23, 38, 34, 46, 50, 12, 23, 0, 33, 20, 24, 20, 15, 11, 14, 25, -1, -15, 21]


In [47]:
# Call the function to run the analysis
run_analysis()

Results saved to: burst_results_analyses.xlsx


## Function to Visualize Soil and Pipe

In [48]:
# 12. Function to visualize the soil and pipe
def visualize_soil_and_pipe(soil_nodes, pipe_nodes, displacement=None):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the soil grid (just plot the nodes)
    soil_x = [node[0] for node in soil_nodes.values()]
    soil_y = [node[1] for node in soil_nodes.values()]
    soil_z = [node[2] for node in soil_nodes.values()]
    
    ax.scatter(soil_x, soil_y, soil_z, color='blue', marker='o', s=20, label='Soil Nodes')

    # Plot the pipe nodes
    pipe_x = [node[0] for node in pipe_nodes]
    pipe_y = [node[1] for node in pipe_nodes]
    pipe_z = [node[2] for node in pipe_nodes]
    
    ax.scatter(pipe_x, pipe_y, pipe_z, color='brown', marker='x', s=50, label='Pipe Nodes')

    # If displacement is provided, plot the displaced pipe nodes
    if displacement is not None:
        displaced_x = [pipe_nodes[i][0] + displacement[i][0] for i in range(len(pipe_nodes))]
        displaced_y = [pipe_nodes[i][1] + displacement[i][1] for i in range(len(pipe_nodes))]
        displaced_z = [pipe_nodes[i][2] + displacement[i][2] for i in range(len(pipe_nodes))]
        
        ax.scatter(displaced_x, displaced_y, displaced_z, color='g', marker='^', s=50, label='Displaced Pipe Nodes')

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.legend()
    plt.show()

In [12]:
# visualize_soil_and_pipe(soil_nodes, pipe_nodes, displacement=None)